In [1]:
import pandas as pd
import numpy as np

In [2]:
list_egid = range(100)

In [3]:
CO2 = {
    
    "mazout" : 0.301,
    "gaz" : 0.228,
    "e_directe" : 0.063,
    "bois" : 0.045,
    "PAC" : 0.063,
}

In [4]:
idc = pd.read_excel('../PRE_PROCESS_DATA/chauffage_besoins.xlsx')
idc.reset_index(inplace=True)
idc["Affectation"] = idc['index']
idc.drop("index", axis=1, inplace=True)
idc["NumAffectation"] = ['I', 'II', 'VI', 'III', 'V', 'IX', 'IX', 'IX', 'X', 'IV', 'VIII', 'XI']

bes_ecs = pd.read_excel('../PRE_PROCESS_DATA/ecs_besoins.xlsx', head='IDC')
for i in idc.columns[:-2]:
    idc[i] += bes_ecs["BesoinECS"]

bes_ecs["NumAffectation"] = ['I', 'II', 'VI', 'III', 'V', 'IX', 'IX', 'IX', 'X', 'IV', 'VIII', 'XI']
bes_ecs

,Affectation,BesoinECS,NumAffectation
0,Maison individuelle,13.9,I
1,Bâtiment collectif,20.8,II
2,Lieux de rassemblement,13.9,VI
3,Administratif,6.9,III
4,Commerce,6.9,V
5,Industrie,6.9,IX
6,Garage,6.9,IX
7,Gare,6.9,IX
8,Dépots,1.4,X
9,École,6.9,IV


In [5]:
data_ren = pd.DataFrame(columns=["EGID"])

for i, e in enumerate(list_egid[:50]):
    data_ren.loc[i, "EGID"] = "BAT{}".format(int(e))

data_ren["EpConstr"] = np.random.choice(["_1918", "1919_1945", "1946_1960", "1961_1970", "1971_1980", "1981_1985", "1986_1990"], size=50)
data_ren["SRE"] = np.random.uniform(low=100, high=2000, size=50)
data_ren["Affect"] = np.random.choice(['I', 'II', 'III', 'IV', 'V', 'VI', 'XI'], size=50)

data_ren["EffAubaine"] = np.random.choice(["yes", "no"], p=[0.05, 0.95], size=50)
data_ren["PerCentSubMin"] = np.random.normal(loc=0.1, scale=0.015, size=50)

data_ren["AgEner"] = np.random.choice(["mazout", "gaz", "e_direct"], size=50)
data_ren["Label"] = np.random.choice(["Minergie", "MinergieP"], p=[0.75, 0.25], size=50)

data_ren["BudgetTotal"] = np.random.normal(loc=800, scale=10, size=50)*data_ren["SRE"]
data_ren["BudgetTotal"] = data_ren["BudgetTotal"].map(int)

sub_can = {"Minergie": 0.09,
          "MinergieP": 0.11}

sub_com = {"Minergie": 0.04,
          "MinergieP": 0.08}

data_ren['BudgetSubCan'] = data_ren['BudgetTotal'].map(int)
data_ren['BudgetSubCom'] = data_ren['BudgetTotal'].map(int)

for i in data_ren.index:
    data_ren.at[i, 'BudgetSubCan'] *= sub_can['MinergieP'] if [data_ren['Label']=="MinergieP"] else data_ren["BudgetSubCan"][i]*sub_can['MinergieP']
    data_ren.at[i, 'BudgetSubCom'] *= sub_com['MinergieP'] if [data_ren['Label']=="MinergieP"] else data_ren["BudgetSubCom"][i]*sub_com['MinergieP']

data_ren["BudgetSubCan"] = data_ren["BudgetSubCan"].map(lambda x: min(20000, x))
data_ren["BudgetSubCom"] = data_ren["BudgetSubCom"].map(lambda x: min(10000, x))
data_ren["BudgetPrivate"] = data_ren["BudgetTotal"] - data_ren["BudgetSubCan"] - data_ren["BudgetSubCom"]


data_ren["IdProjet"] = data_ren["EGID"] + "-REN"

data_ren["BesECS"] = data_ren["SRE"]

#for i in data_ren.index:
#    data_ren.at[i, "BesECS"] = bes_ecs[bes_ecs["NumAffectation"] == data_ren.at[i, "Affect"]] 

In [6]:
data_new = pd.DataFrame(columns=["EGID"])

for i, e in enumerate(list_egid[-20:]):
    data_new.loc[i, "EGID"] = "BAT{}".format(int(e))

data_new["EpConstr"] = np.random.choice(range(2000, 2018), size=20)
data_new["SRE"] = np.random.uniform(low=100, high=2000, size=20)
data_new["Alloc"] = np.random.choice(range(1, 12), size=20)

data_new["EffAubaine"] = np.random.choice(["yes", "no"], p=[0.6, 0.4], size=20)
data_new["PerCentSubMin"] = np.random.normal(loc=0.05, scale=0.002, size=20)

data_new["AgEner"] = np.random.choice(["PAC", "bois"], size=20)

data_new["Label"] = np.random.choice(["Minergie", "MinergieP"], p=[0.6, 0.4], size=20)

data_new["BudgetTotal"] = np.random.normal(loc=3500, scale=200, size=20)*data_new["SRE"]
data_new["BudgetTotal"] = data_new["BudgetTotal"].map(int)

sub_can = {"Minergie": 0.08,
          "MinergieP": 0.10}

sub_com = {"Minergie": 0.03,
          "MinergieP": 0.06}

data_new['BudgetSubCan'] = data_new['BudgetTotal'].map(int)
data_new['BudgetSubCom'] = data_new['BudgetTotal'].map(int)

for i in data_new.index:
    data_new.at[i, 'BudgetSubCan'] *= sub_can['MinergieP'] if [data_new['Label']=="MinergieP"] else data_new["BudgetSubCan"][i]*sub_can['MinergieP']
    data_new.at[i, 'BudgetSubCom'] *= sub_com['MinergieP'] if [data_new['Label']=="MinergieP"] else data_new["BudgetSubCom"][i]*sub_com['MinergieP']

data_new["BudgetSubCan"] = data_new["BudgetSubCan"].map(lambda x: min(50000, x))
data_new["BudgetSubCom"] = data_new["BudgetSubCom"].map(lambda x: min(20000, x))
data_new["BudgetPrivate"] = data_new["BudgetTotal"] - data_new["BudgetSubCan"] - data_new["BudgetSubCom"]

data_new["IdProjet"] = data_new["EGID"] + "-NC"

data_new["kWhEco"] = data_new["SRE"]

for i in data_new.index:
    data_new.at[i, "kWhEco"] *= 15*50 if [data_new['Label']=="MinergieP"] else 5*50

data_new["kgCO2Eco"] = data_new["kWhEco"]

for i in data_new.index:
    data_new.at[i, "kgCO2Eco"] *= CO2[data_new.at[i, "AgEner"]]
    
data_new['kWhEco/CHF_tot'] = data_new['kWhEco'] / data_new['BudgetTotal']
data_new['kWhEco/CHF_subv'] = data_new['kWhEco'] / (data_new['BudgetSubCan']+data_new['BudgetSubCom'])
    
data_new.reset_index(inplace=True, drop=True)

data_new


,EGID,EpConstr,SRE,Alloc,EffAubaine,PerCentSubMin,AgEner,Label,BudgetTotal,BudgetSubCan,BudgetSubCom,BudgetPrivate,IdProjet,kWhEco,kgCO2Eco,kWhEco/CHF_tot,kWhEco/CHF_subv
0,BAT80,2008,858.583582,11,no,0.049108,bois,Minergie,3150222,50000,20000,3080222,BAT80-NC,6.439377e+05,28977.195880,0.204410,9.199110
1,BAT81,2007,230.596530,11,yes,0.049792,PAC,Minergie,763523,50000,20000,693523,BAT81-NC,1.729474e+05,10895.686029,0.226512,2.470677
2,BAT82,2011,916.001386,3,yes,0.051222,PAC,Minergie,3045703,50000,20000,2975703,BAT82-NC,6.870010e+05,43281.065476,0.225564,9.814301
3,BAT83,2006,653.228448,11,yes,0.049221,bois,Minergie,2268344,50000,20000,2198344,BAT83-NC,4.899213e+05,22046.460128,0.215982,6.998876
4,BAT84,2009,1868.147250,3,yes,0.047385,PAC,Minergie,7120593,50000,20000,7050593,BAT84-NC,1.401110e+06,88269.957585,0.196769,20.015863
5,BAT85,2005,1562.724190,3,yes,0.048666,bois,Minergie,5860437,50000,20000,5790437,BAT85-NC,1.172043e+06,52741.941403,0.199992,16.743473
6,BAT86,2016,931.572711,4,yes,0.047207,bois,Minergie,3629286,50000,20000,3559286,BAT86-NC,6.986795e+05,31440.579010,0.192512,9.981136
7,BAT87,2013,1073.772512,7,no,0.051849,bois,Minergie,3380554,50000,20000,3310554,BAT87-NC,8.053294e+05,36239.822278,0.238224,11.504705
8,BAT88,2003,1534.242125,6,yes,0.048040,bois,MinergieP,5360135,50000,20000,5290135,BAT88-NC,1.150682e+06,51780.671724,0.214674,16.438308
9,BAT89,2008,289.588184,5,yes,0.048448,bois,MinergieP,1023864,50000,20000,953864,BAT89-NC,2.171911e+05,9773.601197,0.212129,3.102731


In [7]:
list_idx_cae = np.random.choice(data_ren.index, size=20)

data_cae = data_ren.loc[list_idx_cae, ["EGID", "EpConstr", "SRE", "Alloc"]]

data_cae2 = pd.DataFrame(columns=["EGID"])
for i, e in enumerate(list_egid[50:80]):
    data_cae2.loc[i, "EGID"] = "BAT{}".format(int(e))

data_cae2["EpConstr"] = np.random.choice(range(1930, 2000), size=30)
data_cae2["SRE"] = np.random.uniform(low=100, high=2000, size=30)
data_cae2["Alloc"] = np.random.choice(range(1, 12), size=30)

data_cae = pd.concat([data_cae, data_cae2])
data_cae = data_cae.reindex()

data_cae["EffAubaine"] = np.random.choice(["yes", "no"], p=[0.2, 0.8], size=50)
data_cae["PerCentSubMin"] = np.random.normal(loc=0.2, scale=0.05, size=50)

data_cae["AgEner_0"] = np.random.choice(["mazout", "gaz", "e_direct"], size=50)
data_cae["AgEner_1"] = np.random.choice(["PAC", "bois"], size=50)

data_cae["BudgetTotal"] = np.random.normal(loc=80, scale=2, size=50)*data_cae["SRE"]
data_cae["BudgetTotal"] = data_cae["BudgetTotal"].map(int)


data_cae.reset_index(inplace=True, drop=True)

In [8]:
change = [(u[0], u[1]) for u in np.array(np.meshgrid(
    ["mazout", "gaz", "e_direct"], 
    ["PAC", "bois"])).T.reshape(-1,2)]
change

sub_can = {t: (0.20 if t[1] == "PAC" else 0.1) for t in change}

sub_com = {t: (0.30 if t[1] == "bois" else 0.1) for t in change}

data_cae["BudgetSubCan"] = [sub_can[(row[6], row[7])]*row[8] for _, row in data_cae.iterrows()]
data_cae["BudgetSubCom"] = [sub_com[(row[6], row[7])]*row[8] for _, row in data_cae.iterrows()]

data_cae["BudgetSubCan"] = data_cae["BudgetSubCan"].map(lambda x: min(5000, x))
data_cae["BudgetSubCom"] = data_cae["BudgetSubCom"].map(lambda x: min(5000, x))
data_cae["BudgetPrivate"] = data_cae["BudgetTotal"] - data_cae["BudgetSubCan"] - data_cae["BudgetSubCom"]

data_cae["IdProjet"] = data_cae["EGID"] + "-CAE"

data_cae["EstCons"] = data_cae["SRE"]

data_cae.reset_index(inplace=True, drop=True)

data_cae.head()

,EGID,EpConstr,SRE,Alloc,EffAubaine,PerCentSubMin,AgEner_0,AgEner_1,BudgetTotal,BudgetSubCan,BudgetSubCom,BudgetPrivate,IdProjet,EstCons
0,BAT15,1961_1970,1163.265128,NaN,no,0.180868,e_direct,PAC,91854,5000.0,5000.0,81854.0,BAT15-CAE,1163.265128
1,BAT0,1971_1980,259.329140,NaN,no,0.113344,gaz,bois,20764,2076.4,5000.0,13687.6,BAT0-CAE,259.329140
2,BAT15,1961_1970,1163.265128,NaN,yes,0.279087,e_direct,PAC,93792,5000.0,5000.0,83792.0,BAT15-CAE,1163.265128
3,BAT30,1971_1980,1400.556241,NaN,no,0.128720,gaz,PAC,113780,5000.0,5000.0,103780.0,BAT30-CAE,1400.556241
4,BAT17,_1918,1622.928478,NaN,no,0.180353,gaz,bois,125288,5000.0,5000.0,115288.0,BAT17-CAE,1622.928478


In [9]:
#nodes = [["total_CAE", "total_NEW", "total_REN"], ["private", "canton", "commune"], ["useful", "dead_weight"], ['bois', 'PAC', 'NC Minergie', 'NC MinergieP', 'REN Minergie', 'REN MinergieP']]
nodes = [[0, 1, 2], [3, 4, 5], [6, 7], [8, 9, 10, 11, 12, 13]]

In [10]:
flows = []

for _, row in data_cae.iterrows():
    flows.append({"SRC": nodes[0][0], "TAR":nodes[1][0], "VAL": row["BudgetPrivate"], "LAB": row['IdProjet']})
    flows.append({"SRC": nodes[0][0], "TAR":nodes[1][1], "VAL": row["BudgetSubCan"], "LAB": row['IdProjet']})
    flows.append({"SRC": nodes[0][0], "TAR":nodes[1][2], "VAL": row["BudgetSubCom"], "LAB": row['IdProjet']})
    
    if row["EffAubaine"] == "no":
        flows.append({"SRC": nodes[1][0], "TAR":nodes[2][0], "VAL": row["BudgetPrivate"], "LAB": row['IdProjet']})
        flows.append({"SRC": nodes[1][1], "TAR":nodes[2][0], "VAL": row["BudgetSubCan"], "LAB": row['IdProjet']})
        flows.append({"SRC": nodes[1][2], "TAR":nodes[2][0], "VAL": row["BudgetSubCom"], "LAB": row['IdProjet']})
    
        if row['AgEner_1'] == 'bois':
            flows.append({"SRC": nodes[2][0], "TAR":nodes[3][0], "VAL": row["BudgetTotal"], "LAB": row['IdProjet']})
        
        if row['AgEner_1'] == 'PAC':
            flows.append({"SRC": nodes[2][0], "TAR":nodes[3][1], "VAL": row["BudgetTotal"], "LAB": row['IdProjet']})
        
    if row["EffAubaine"] == "yes":
        flows.append({"SRC": nodes[1][0], "TAR":nodes[2][1], "VAL": row["BudgetPrivate"], "LAB": row['IdProjet']})
        flows.append({"SRC": nodes[1][1], "TAR":nodes[2][1], "VAL": row["BudgetSubCan"], "LAB": row['IdProjet']})
        flows.append({"SRC": nodes[1][2], "TAR":nodes[2][1], "VAL": row["BudgetSubCom"], "LAB": row['IdProjet']})
        
        if row['AgEner_1'] == 'bois':
            flows.append({"SRC": nodes[2][1], "TAR":nodes[3][0], "VAL": row["BudgetTotal"], "LAB": row['IdProjet']})

        if row['AgEner_1'] == 'PAC':
            flows.append({"SRC": nodes[2][1], "TAR":nodes[3][1], "VAL": row["BudgetTotal"], "LAB": row['IdProjet']})
        
for _, row in data_new.iterrows():
    flows.append({"SRC": nodes[0][1], "TAR":nodes[1][0], "VAL": row["BudgetPrivate"], "LAB": row['IdProjet']})
    flows.append({"SRC": nodes[0][1], "TAR":nodes[1][1], "VAL": row["BudgetSubCan"], "LAB": row['IdProjet']})
    flows.append({"SRC": nodes[0][1], "TAR":nodes[1][2], "VAL": row["BudgetSubCom"], "LAB": row['IdProjet']})
    
    if row["EffAubaine"] == "no":
        flows.append({"SRC": nodes[1][0], "TAR":nodes[2][0], "VAL": row["BudgetPrivate"], "LAB": row['IdProjet']})
        flows.append({"SRC": nodes[1][1], "TAR":nodes[2][0], "VAL": row["BudgetSubCan"], "LAB": row['IdProjet']})
        flows.append({"SRC": nodes[1][2], "TAR":nodes[2][0], "VAL": row["BudgetSubCom"], "LAB": row['IdProjet']})
        
        if row['Label'] == 'Minergie':
            flows.append({"SRC": nodes[2][0], "TAR":nodes[3][2], "VAL": row["BudgetTotal"], "LAB": row['IdProjet']})
        
        if row['Label'] == 'MinergieP':
            flows.append({"SRC": nodes[2][0], "TAR":nodes[3][3], "VAL": row["BudgetTotal"], "LAB": row['IdProjet']})
    
    if row["EffAubaine"] == "yes":
        flows.append({"SRC": nodes[1][0], "TAR":nodes[2][1], "VAL": row["BudgetPrivate"], "LAB": row['IdProjet']})
        flows.append({"SRC": nodes[1][1], "TAR":nodes[2][1], "VAL": row["BudgetSubCan"], "LAB": row['IdProjet']})
        flows.append({"SRC": nodes[1][2], "TAR":nodes[2][1], "VAL": row["BudgetSubCom"], "LAB": row['IdProjet']})
    
        if row['Label'] == 'Minergie':
            flows.append({"SRC": nodes[2][1], "TAR":nodes[3][2], "VAL": row["BudgetTotal"], "LAB": row['IdProjet']})
        
        if row['Label'] == 'MinergieP':
            flows.append({"SRC": nodes[2][1], "TAR":nodes[3][3], "VAL": row["BudgetTotal"], "LAB": row['IdProjet']})
        
for _, row in data_ren.iterrows():
    flows.append({"SRC": nodes[0][2], "TAR":nodes[1][0], "VAL": row["BudgetPrivate"], "LAB": row['IdProjet']})
    flows.append({"SRC": nodes[0][2], "TAR":nodes[1][1], "VAL": row["BudgetSubCan"], "LAB": row['IdProjet']})
    flows.append({"SRC": nodes[0][2], "TAR":nodes[1][2], "VAL": row["BudgetSubCom"], "LAB": row['IdProjet']})
    
    if row["EffAubaine"] == "no":
        flows.append({"SRC": nodes[1][0], "TAR":nodes[2][0], "VAL": row["BudgetPrivate"], "LAB": row['IdProjet']})
        flows.append({"SRC": nodes[1][1], "TAR":nodes[2][0], "VAL": row["BudgetSubCan"], "LAB": row['IdProjet']})
        flows.append({"SRC": nodes[1][2], "TAR":nodes[2][0], "VAL": row["BudgetSubCom"], "LAB": row['IdProjet']})
    
        if row['Label'] == 'Minergie':
            flows.append({"SRC": nodes[2][0], "TAR":nodes[3][4], "VAL": row["BudgetTotal"], "LAB": row['IdProjet']})

        if row['Label'] == 'MinergieP':
            flows.append({"SRC": nodes[2][0], "TAR":nodes[3][5], "VAL": row["BudgetTotal"], "LAB": row['IdProjet']})
            
    if row["EffAubaine"] == "yes":
        flows.append({"SRC": nodes[1][0], "TAR":nodes[2][1], "VAL": row["BudgetPrivate"], "LAB": row['IdProjet']})
        flows.append({"SRC": nodes[1][1], "TAR":nodes[2][1], "VAL": row["BudgetSubCan"], "LAB": row['IdProjet']})
        flows.append({"SRC": nodes[1][2], "TAR":nodes[2][1], "VAL": row["BudgetSubCom"], "LAB": row['IdProjet']})
        
        if row['Label'] == 'Minergie':
            flows.append({"SRC": nodes[2][1], "TAR":nodes[3][4], "VAL": row["BudgetTotal"], "LAB": row['IdProjet']})

        if row['Label'] == 'MinergieP':
            flows.append({"SRC": nodes[2][1], "TAR":nodes[3][5], "VAL": row["BudgetTotal"], "LAB": row['IdProjet']})
    

In [11]:
source = []
target = []
value = []
label = []

labels = ["Chang Ag Éner", "Nouvelles Cons", "Rénovations", "Investissements Privés", "Subv Cantonale", "Subv Communale", "Encouragement Réussi", "Aubaine", 'bois', 'PAC', 'NC Minergie', 'NC MinergieP', 'REN Minergie', 'REN MinergieP']
colors = ["White", "White", "White", "White", "White", "White", "White", "White", 'White', 'White', 'White', 'White', 'White', 'White']


for i, _ in enumerate(flows):
    source.append(flows[i]['SRC'])
    target.append(flows[i]['TAR'])
    value.append(flows[i]['VAL'])
    label.append(flows[i]['LAB'])

In [12]:
import plotly.plotly as py
import plotly.graph_objs as go

In [14]:
data = dict(
    type = 'sankey',
    opacity =0.5,
    node = dict(
        pad = 15,
        thickness = 20,
        line = dict(
            color = "black",
            width = 1),
        label = labels,
        color = colors),
    link = dict(
        source = source,
        target = target,
        value = value,
        label = label,
        color = 'lightgray')
)

layout =  dict(
    title = "INVESTMENTS SANKEY DIAGRAM",
    font = dict(
        size = 12)
)

fig = dict(data=[data], layout=layout)
py.iplot(fig, validate=True)